In [343]:
#-*- coding : utf-8 -*-
# coding: utf-8
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM

## 加载数据

In [344]:
text = ''
with open('Poem.txt', 'r', encoding='gb18030') as fr:   # gb18030兼容中文比较好
    for line in fr:
        if '第' in line or '◎' in line:  # 删除冗余信息，只保留古诗内容。
            text += '+'
        else:
            text += line.strip()
print(text[:10000])

++秦川雄帝宅，函谷壮皇居。绮殿千寻起，离宫百雉馀。连薨遥接汉，飞观迥凌虚。云日隐层阙，风烟出绮疏。岩廊罢机务，崇文聊驻辇。玉匣启龙图，金绳披凤篆。韦编断仍续，缥帙舒还卷。对此乃淹留，欹案观坟典。移步出词林，停舆欣武宴。雕弓写明月，骏马疑流电。惊雁落虚弦，啼猿悲急箭。阅赏诚多美，于兹乃忘倦。鸣笳临乐馆，眺听欢芳节。急管韵朱弦，清歌凝白雪。彩凤肃来仪，玄鹤纷成列。去兹郑卫声，雅音方可悦。芳辰追逸趣，禁苑信多奇。桥形通汉上，峰势接云危。烟霞交隐映，花鸟自参差。何如肆辙迹，万里赏瑶池。飞盖去芳园，兰桡游翠渚。萍间日彩乱，荷处香风举。桂楫满中川，弦歌振长屿。岂必汾河曲，方为欢宴所。落日双阙昏，回舆九重暮。长烟散初碧，皎月澄轻素。搴幌玩琴书，开轩引云雾。斜汉耿层阁，清风摇玉树。欢乐难再逢，芳辰良可惜。玉酒泛云罍，兰殽陈绮席。千钟合尧禹，百兽谐金石。得志重寸阴，忘怀轻尺璧。建章欢赏夕，二八尽妖妍。罗绮昭阳殿，芬芳玳瑁筵。佩移星正动，扇掩月初圆。无劳上悬圃，即此对神仙。以兹游观极，悠然独长想。披卷览前踪，抚躬寻既往。望古茅茨约，瞻今兰殿广。人道恶高危，虚心戒盈荡。奉天竭诚敬，临民思惠养。纳善察忠谏，明科慎刑赏。六五诚难继，四三非易仰。广待淳化敷，方嗣云亭响。+塞外悲风切，交河冰已结。瀚海百重波，阴山千里雪。迥戍危烽火，层峦引高节。悠悠卷旆旌，饮马出长城。寒沙连骑迹，朔吹断边声。胡尘清玉塞，羌笛韵金钲。绝漠干戈戢，车徒振原隰。都尉反龙堆，将军旋马邑。扬麾氛雾静，纪石功名立。荒裔一戎衣，灵台凯歌入。+执契静三边，持衡临万姓。玉彩辉关烛，金华流日镜。无为宇宙清，有美璇玑正。皎佩星连景，飘衣云结庆。戢武耀七德，升文辉九功。烟波澄旧碧，尘火息前红。霜野韬莲剑，关城罢月弓。钱缀榆天合，新城柳塞空。花销葱岭雪，縠尽流沙雾。秋驾转兢怀，春冰弥轸虑。书绝龙庭羽，烽休凤穴戍。衣宵寝二难，食旰餐三惧。翦暴兴先废，除凶存昔亡。圆盖归天壤，方舆入地荒。孔海池京邑，双河沼帝乡。循躬思励己，抚俗愧时康。元首伫盐梅，股肱惟辅弼。羽贤崆岭四，翼圣襄城七。浇俗庶反淳，替文聊就质。已知隆至道，共欢区宇一。+条风开献节，灰律动初阳。百蛮奉遐赆，万国朝未央。虽无舜禹迹，幸欣天地康。车轨同八表，书文混四方。赫奕俨冠盖，纷纶盛服章。羽旄飞驰道，钟鼓震岩廊。组练辉霞色，霜戟耀朝光。晨宵怀至理，终愧抚遐荒。+寿丘惟旧迹，酆邑乃前

## 创建字典

In [345]:
words = sorted(list(text))
counted_words = {}
for word in words:
    if word in counted_words:
        counted_words[word] += 1
    else:
        counted_words[word] = 1
        

# 去掉低频的字
erase = []
for key in counted_words:
    if counted_words[key] <= 5:
        erase.append(key)
print(len(erase))
print(len(text))
for key in erase:
    text = text.replace(key, '')
    del counted_words[key]

print(text[:10000])
characters = sorted(list(set(text)))
print(len(text))
n_to_char = {n:char for n, char in enumerate(characters)}
char_to_n = {char:n for n, char in enumerate(characters)}

2180
3114114
++秦川雄帝宅，函谷壮皇居。绮殿千寻起，离宫百雉馀。连薨遥接汉，飞观迥凌虚。云日隐层阙，风烟出绮疏。岩廊罢机务，崇文聊驻辇。玉匣启龙图，金绳披凤篆。韦编断仍续，缥帙舒还卷。对此乃淹留，欹案观坟典。移步出词林，停舆欣武宴。雕弓写明月，骏马疑流电。惊雁落虚弦，啼猿悲急箭。阅赏诚多美，于兹乃忘倦。鸣笳临乐馆，眺听欢芳节。急管韵朱弦，清歌凝白雪。彩凤肃来仪，玄鹤纷成列。去兹郑卫声，雅音方可悦。芳辰追逸趣，禁苑信多奇。桥形通汉上，峰势接云危。烟霞交隐映，花鸟自参差。何如肆辙迹，万里赏瑶池。飞盖去芳园，兰桡游翠渚。萍间日彩乱，荷处香风举。桂楫满中川，弦歌振长屿。岂必汾河曲，方为欢宴所。落日双阙昏，回舆九重暮。长烟散初碧，皎月澄轻素。搴幌玩琴书，开轩引云雾。斜汉耿层阁，清风摇玉树。欢乐难再逢，芳辰良可惜。玉酒泛云罍，兰殽陈绮席。千钟合尧禹，百兽谐金石。得志重寸阴，忘怀轻尺璧。建章欢赏夕，二八尽妖妍。罗绮昭阳殿，芬芳玳瑁筵。佩移星正动，扇掩月初圆。无劳上悬圃，即此对神仙。以兹游观极，悠然独长想。披卷览前踪，抚躬寻既往。望古茅茨约，瞻今兰殿广。人道恶高危，虚心戒盈荡。奉天竭诚敬，临民思惠养。纳善察忠谏，明科慎刑赏。六五诚难继，四三非易仰。广待淳化敷，方嗣云亭响。+塞外悲风切，交河冰已结。瀚海百重波，阴山千里雪。迥戍危烽火，层峦引高节。悠悠卷旆旌，饮马出长城。寒沙连骑迹，朔吹断边声。胡尘清玉塞，羌笛韵金钲。绝漠干戈戢，车徒振原隰。都尉反龙堆，将军旋马邑。扬麾氛雾静，纪石功名立。荒裔一戎衣，灵台凯歌入。+执契静三边，持衡临万姓。玉彩辉关烛，金华流日镜。无为宇宙清，有美璇玑正。皎佩星连景，飘衣云结庆。戢武耀七德，升文辉九功。烟波澄旧碧，尘火息前红。霜野韬莲剑，关城罢月弓。钱缀榆天合，新城柳塞空。花销葱岭雪，縠尽流沙雾。秋驾转兢怀，春冰弥轸虑。书绝龙庭羽，烽休凤穴戍。衣宵寝二难，食旰餐三惧。翦暴兴先废，除凶存昔亡。圆盖归天壤，方舆入地荒。孔海池京邑，双河沼帝乡。循躬思励己，抚俗愧时康。元首伫盐梅，股肱惟辅弼。羽贤崆岭四，翼圣襄城七。浇俗庶反淳，替文聊就质。已知隆至道，共欢区宇一。+条风开献节，灰律动初阳。百蛮奉遐赆，万国朝未央。虽无舜禹迹，幸欣天地康。车轨同八表，书文混四方。赫奕俨冠盖，纷纶盛服章。羽旄飞驰道，钟鼓震岩廊。组练辉霞色，霜戟耀朝光。晨宵怀至理，终愧抚遐

In [346]:
print(len(characters))
print(characters[:640])

5363
['(', ')', '+', '-', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '“', '”', '□', '、', '。', '々', '《', '》', '䴔', '䴖', '一', '丁', '七', '万', '丈', '三', '上', '下', '不', '与', '丑', '专', '且', '丕', '世', '丘', '丙', '业', '丛', '东', '丝', '丞', '两', '严', '丧', '个', '中', '丰', '丱', '串', '临', '丸', '丹', '为', '主', '丽', '举', '乂', '乃', '久', '么', '义', '之', '乌', '乍', '乎', '乏', '乐', '乔', '乖', '乘', '乙', '九', '乞', '也', '习', '乡', '书', '买', '乱', '乳', '乾', '了', '予', '争', '事', '二', '于', '亏', '云', '互', '五', '井', '亘', '亚', '些', '亟', '亡', '亢', '交', '亥', '亦', '产', '亨', '亩', '享', '京', '亭', '亮', '亲', '亸', '亹', '人', '亿', '什', '仁', '仄', '仅', '仆', '仇', '今', '介', '仍', '从', '仑', '仓', '仔', '仕', '他', '仗', '付', '仙', '仝', '仞', '代', '令', '以', '仪', '仰', '仲', '价', '任', '仿', '企', '伉', '伊',

## 构造训练集合  
X为训练序列  
Y为训练序列之后的特定字符

In [381]:
X = []
Y = []
length = int(len(text)/10)
print(length)
seq_length = 5
for i in range(0, length-seq_length, 1):
    sequence = text[i:i + seq_length]
    label =text[i + seq_length]
    if '+' not in sequence and char_to_n[label] != '+' :  # 存在‘+’说明是两首诗，舍弃该数据即可。
        X.append([char_to_n[char] for char in sequence])
        Y.append(char_to_n[label])


310940


## 转换成LSTM能够处理的格式

In [382]:
X_modified = np.reshape(X, (len(X), seq_length, 1))
X_modified = X_modified / float(len(characters))
print(X_modified.shape)
print(len(set(Y)))
print(len(Y))
print(max(Y))

(289696, 5, 1)
4588
289696
5362


## LSTM建模

In [383]:
model = Sequential()
model.add(LSTM(512, input_shape=(X_modified.shape[1],  X_modified.shape[2]), return_sequences=True))
model.add(Dropout(0.25))
model.add(LSTM(256))
model.add(Dropout(0.6))
model.add(Dense(max(Y) + 1, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

## 训练模型

In [ ]:
epochs = 15
batch_size= 128
x_train = X_modified
y_train = Y
history = model.fit(
          x_train, 
          y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_split = 0.2,
          # 训练集的20%用作验证集
        )

Train on 231756 samples, validate on 57940 samples
Epoch 1/15
221952/231756 [===========================>..] - ETA: 12s - loss: 6.6539